在Pytorch中训练模型并为其进行损失与精度校验

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook
from torch.nn import functional as F

In [12]:
# 定一个MLP模型
class MLP(nn.Module):
    def __init__(self,input_dim,hidden_dim,num_classes) -> None:
        super(MLP,self).__init__()
        self.linear1 = nn.Linear(input_dim,hidden_dim)
        self.relu = F.relu
        self.linear2 = nn.Linear(hidden_dim,num_classes)
    def forward(self,inputs):
        hidden = self.linear1(inputs.to(torch.float32))
        relu = self.relu(hidden)
        output = self.linear2(relu)
        log_probs = F.log_softmax(output)
        return log_probs

In [13]:
# 初始化CUDA设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cuda'

In [14]:
# 初始化一个异或问题张量
x_train = torch.tensor([[0,0],[0,1],[1,0],[1,1]],device=device)
# 初始化一个异或问题标签张量
y_train = torch.tensor([[0],[1],[1],[0]],device=device)

In [15]:
x_train.dtype

torch.int64

In [16]:
y_train.dtype

torch.int64

In [17]:
# 创建多层感知器模型，输入层大小为2，隐含层大小为5，类别数为2
model = MLP(input_dim=2,hidden_dim=5,num_classes=2)
model

MLP(
  (linear1): Linear(in_features=2, out_features=5, bias=True)
  (linear2): Linear(in_features=5, out_features=2, bias=True)
)

In [18]:
# 使用对数softmax函数(log_softmax)作为输出时，需调用负数对数似然损失
criterion = nn.NLLLoss()

In [38]:
# 调用随机梯度下降优化器
# 优化器调用参数分别为：
# - 参数：来自于模型，调用model.parameters()获得
# - 学习率：指定模型的学习速率
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [39]:
# 将模型转移到GPU
model.to(device=device)

MLP(
  (linear1): Linear(in_features=2, out_features=5, bias=True)
  (linear2): Linear(in_features=5, out_features=2, bias=True)
)

In [40]:
with tqdm_notebook(range(500),desc="Starting") as bar:
    for i in bar:
        # 执行预测
        y_pred = model(x_train)
        # 获取损失量
        loss = criterion(y_pred,y_train.squeeze(dim=1))
        # 清空优化器的梯度信息，以防止积累
        optimizer.zero_grad()
        bar.set_postfix(loss="{:.2f}".format(loss))
        # 反向传播损失函数计算的梯度值
        loss.backward()
        optimizer.step()
        # 更新
        bar.update()
print("Parameters: ")
for name,param in model.named_parameters():
    print(name,param)
final_pred = model(x_train)
print("Predict Results are : ",final_pred.argmax(axis=1))

/tmp/ipykernel_3717/4138451745.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm_notebook(range(500),desc="Starting") as bar:


Starting:   0%|          | 0/500 [00:00<?, ?it/s]

/tmp/ipykernel_3717/2451156915.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs = F.log_softmax(output)


Parameters: 
linear1.weight Parameter containing:
tensor([[-1.6305,  1.6312],
        [ 1.8609,  0.2028],
        [-0.2100, -0.3954],
        [ 1.9139,  1.9144],
        [-1.1447, -1.1452]], device='cuda:0', requires_grad=True)
linear1.bias Parameter containing:
tensor([-9.6416e-04,  2.0105e-04, -6.6870e-02, -1.9134e+00,  1.1455e+00],
       device='cuda:0', requires_grad=True)
linear2.weight Parameter containing:
tensor([[-1.7619, -1.5393, -0.2899,  2.3601,  1.3240],
        [ 1.2332,  0.9601,  0.0979, -2.1523, -1.2707]], device='cuda:0',
       requires_grad=True)
linear2.bias Parameter containing:
tensor([ 0.1792, -0.6557], device='cuda:0', requires_grad=True)
Predict Results are :  tensor([0, 1, 1, 0], device='cuda:0')
